In [30]:
import pandas as pd
import numpy as np
import re
import os

In [4]:
dataFileDF = pd.DataFrame([{'fondo':'A', 'fileName':'../data/vcfA2020-2020.csv'},
                           {'fondo':'B', 'fileName':'../data/vcfB2020-2020.csv'},
                           {'fondo':'C', 'fileName':'../data/vcfC2020-2020.csv'},
                           {'fondo':'D', 'fileName':'../data/vcfD2020-2020.csv'},
                           {'fondo':'E', 'fileName':'../data/vcfE2020-2020.csv'}])
dataFileDF.head()

,fondo,fileName
0,A,../data/vcfA2020-2020.csv
1,B,../data/vcfB2020-2020.csv
2,C,../data/vcfC2020-2020.csv
3,D,../data/vcfD2020-2020.csv
4,E,../data/vcfE2020-2020.csv


In [32]:
# Creamos la carpeta results para guardar el output

if not os.path.exists('../results'):
    os.mkdir('../results')

In [36]:
reHeader = re.compile(r'^Fecha.*')
reData = re.compile(r'^\d{4}-\d{2}-\d{2}.*')
                     
for index, row in dataFileDF.iterrows():
    if index>0:
        continue
    print(row['fileName'])
    
    dataRaw = []
    with open(row['fileName'], 'r') as f:
        for line in f:
            #print(line)
            matchHeader = reHeader.match(line)
            matchData = reData.match(line)
            if matchHeader:
                headerRaw = matchHeader.group(0)
            if matchData:
                dataRaw.append(matchData.group(0))
                
    with open('../results/dataClean.csv', 'w') as f:
        f.write(headerRaw+'\n')
        for line in dataRaw:
            f.write(line+'\n')

../data/vcfA2020-2020.csv
